In [37]:
import os
import sys
import traci
import pandas as pd
import numpy as np
from itertools import product
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from xml.dom import minidom

In [1]:
csv_file = '.\demand.csv'
data =pd.read_csv(csv_file)
data =pd.DataFrame(data)
data =data[data['end']<=10800]
data['type']="type1"
data = data.sort_values(by='begin') 
print(data.head())
#####################################
#demand file generated (sumo) using traffic count data 0=7:00AM
#     id     begin  end   from to    vehsPerHour type
#      1        0  3600     1  20         86.0  type1
#      2        0  3600     1 -33          5.0  type1
#####################################

In [4]:
#generating demand file for krauss model
def df_to_xml(df):
    root = ET.Element("routes")
    root.set("xmlns:xsi", "http://www.w3.org/2001/XMLSchema-instance")
    root.set("xsi:noNamespaceSchemaLocation", "http://sumo.dlr.de/xsd/routes_file.xsd")
    vtype_1 = ET.SubElement(root, "vType", id="type1", length="5", minGap="2.5", maxSpeed="13.4112",
                            vClass="passenger", accel="2.6", decel="4.5", emergencyDecel="9", carFollowModel="KraussOrig1",
                            sigma="0.5", tau="1", sigmaStep="1", speedDev="0.1", emissionClass="PHEMlight/PC_G_EU4")

    for index, row in df.iterrows():
        flow = ET.SubElement(root, "flow")
        flow.set("id", str(row['id']))
        flow.set("begin", str(row['begin']))
        flow.set("from", str(row['from']))
        flow.set("to", str(row['to']))
        flow.set("end", str(row['end']))
        flow.set("vehsPerHour", str(row['vehsPerHour']))
        flow.set("type", str(row['type']))

    tree = ET.ElementTree(root)

    xml_str = minidom.parseString(ET.tostring(root)).toprettyxml(indent="  ")

    return xml_str

xml_list = df_to_xml(data) 

In [6]:
file_path = ".\first_xml_output_krauss.xml"

with open(file_path, "w") as xml_file:
    xml_file.write(xml_list[0])

In [5]:
os.environ["SUMO_HOME"] = "C:/Program Files (x86)/Eclipse/Sumo/"
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please set the environment variable 'SUMO_HOME'")

sumo_binary = "C:/Program Files (x86)/Eclipse/Sumo/bin/sumo"
sumocfg_file = "configuration.sumocfg"

traci.start([sumo_binary, "-c", sumocfg_file])

try:
    for step in range(10800):
        traci.simulationStep()

finally:
    traci.close()

In [6]:
#traci.close()

In [2]:
tree = ET.parse('tripinfos.xml')
root = tree.getroot()
ids = []
departs = []
departedge = []
arrivaledge = []

for tripinfo in root.findall('.//tripinfo'):
    ids.append(tripinfo.get('id'))
    departs.append(float(tripinfo.get('depart')))
    departedge.append(tripinfo.get('departLane'))
    arrivaledge.append(tripinfo.get('arrivalLane'))

df = pd.DataFrame({
    'id': ids,
    'depart': departs,
    'departEdge': departedge,
    'arrivalEdge': arrivaledge
})

df = df.sort_values(by='depart')  
print(df)


In [ ]:
df[['from', 'departLane']] = df['departEdge'].str.split('_', expand=True)
df.drop(columns=['departEdge'], inplace=True)
df[['to', 'arrivalLane']] = df['arrivalEdge'].str.split('_', expand=True)
df.drop(columns=['arrivalEdge'], inplace=True)
df['type']='type1'
#tripinfo data used for all three model
df.to_csv('synthetic_tripinfo_data.csv', index=False)
print(df)

In [ ]:
############synthetic_tripinfo_data###################
#id    depart    from    departLane    to    arrivalLane    type
#1       0        -51        1         49        1          type1        
#2       0        -69        1         -1        1          type1
######################################################